<a href="https://colab.research.google.com/github/SrWilson89/049EJ26/blob/main/11_practica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX

Downloading...
From: https://drive.google.com/uc?id=1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX
To: /content/data_berka.db
100% 70.2M/70.2M [00:00<00:00, 114MB/s]


In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%sql sqlite:///data_berka.db


In [ ]:
%%sql
SELECT name, type
from sqlite_master
where type = "table"

 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


In [ ]:
%sql SELECT * FROM CLIENT LIMIT 5

 * sqlite:///data_berka.db
Done.


client_id,birth_number,district_id
1,706213,18
2,450204,1
3,406009,1
4,561201,5
5,605703,5


2.2 liste todos los clientes con cuentas activas y prestamos, incluyendo dinero prestado y saldos de cuenta

In [24]:
%%sql
SELECT
  c.client_id,
  c.district_id, -- ID del distrito asociado al cliente
  a.account_id,
  a.frequency AS account_statement_frequency, -- 'frequency' es una columna real en ACCOUNT
  l.amount AS loan_amount,
  l.status AS loan_status
FROM
  CLIENT AS c
JOIN
  DISPOSITION AS d
  ON c.client_id = d.client_id
JOIN
  ACCOUNT AS a
  ON d.account_id = a.account_id
JOIN
  LOAN AS l
  ON a.account_id = l.account_id
WHERE
  d.type = 'OWNER' AND l.status IN ('A', 'B') -- Filtra por clientes que son propietarios y préstamos activos
LIMIT 10; -- Esto limita los resultados a los primeros 10

 * sqlite:///data_berka.db
Done.


client_id,district_id,account_id,account_statement_frequency,loan_amount,loan_status
2,1,2,monthly,80952,A
25,21,19,monthly,30276,B
31,68,25,monthly,30276,A
78,16,67,monthly,165960,A
116,74,97,monthly,102876,A
158,40,132,after_tr,88440,A
210,66,173,monthly,104808,A
215,71,176,weekly,27456,A
352,56,290,monthly,123696,A
424,1,347,monthly,187224,B


2.3) Identifique clientes con múltiples cuentas.

In [25]:
%%sql
SELECT
  d.client_id,
  COUNT(DISTINCT d.account_id) AS number_of_accounts
FROM
  DISPOSITION AS d
WHERE
  d.type = 'OWNER' -- Asegura que solo contamos las cuentas donde el cliente es el propietario
GROUP BY
  d.client_id
HAVING
  COUNT(DISTINCT d.account_id) > 1; -- Filtra para mostrar solo los clientes con más de una cuenta

 * sqlite:///data_berka.db
Done.


client_id,number_of_accounts


2.4) Recupere transacciones del año 1998, ordenadas por cantidad de transacción, e identifique las 10 transacciones más grandes.

In [27]:
%%sql
SELECT
  trans_id,
  account_id,
  date,
  type,
  operation,
  amount,
  k_symbol
FROM
  TRANS -- Corrected table name from TRAN to TRANS
WHERE
  SUBSTR(date, 1, 2) = '98' -- Assumes date format 'YYMMDD'
ORDER BY
  amount DESC
LIMIT 10;

 * sqlite:///data_berka.db
Done.


trans_id,account_id,date,type,operation,amount,k_symbol
291346,998,981215,VYDAJ,VYBER,87400.0,None
1306770,4462,980606,VYDAJ,VYBER,87300.0,None
1207200,4130,980619,VYDAJ,VYBER,85800.0,None
1170721,4014,980615,VYDAJ,VYBER,81900.0,None
706371,2419,980610,VYDAJ,VYBER,80100.0,None
141211,472,980625,VYDAJ,VYBER,78300.0,None
1155373,3953,980618,VYDAJ,VYBER,78000.0,None
623347,2126,980612,VYDAJ,VYBER,77400.0,None
768915,2625,980620,VYDAJ,VYBER,77100.0,None
816974,2782,980621,VYDAJ,VYBER,76800.0,None


2.5) ¿Cuál es el saldo promedio de cuentas con préstamos en comparación con aquellas sin préstamos?.

In [28]:
%%sql

WITH AccountBalances AS (
  -- CTE (Common Table Expression) para calcular el saldo de cada cuenta
  -- Sumamos las transacciones: 'PRIJEM' (ingreso) suma, 'VYDAJ' (retiro) resta.
  SELECT
    t.account_id,
    COALESCE(SUM(CASE
                    WHEN t.type = 'PRIJEM' THEN t.amount
                    WHEN t.type = 'VYDAJ' THEN -t.amount
                    ELSE 0 -- En caso de otros tipos de transacciones no previstos
                  END), 0) AS current_balance
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
)
SELECT
  CASE
    WHEN L.loan_id IS NOT NULL THEN 'Cuentas CON Préstamos'
    ELSE 'Cuentas SIN Préstamos'
  END AS tipo_cuenta,
  AVG(AB.current_balance) AS saldo_promedio
FROM
  ACCOUNT AS A
LEFT JOIN
  LOAN AS L
  ON A.account_id = L.account_id
LEFT JOIN
  AccountBalances AS AB
  ON A.account_id = AB.account_id
GROUP BY
  tipo_cuenta;

 * sqlite:///data_berka.db
Done.


tipo_cuenta,saldo_promedio
Cuentas CON Préstamos,165840.9760997068
Cuentas SIN Préstamos,76650.47171293882


2.6) Identifique clientes que hayan realizado más de 10 transacciones en el año 1998 específico pero que no hayan solicitado un préstamo.

In [29]:
%%sql

SELECT
  c.client_id,
  COUNT(DISTINCT t.trans_id) AS total_transactions_1998
FROM
  CLIENT AS c
JOIN
  DISPOSITION AS d
  ON c.client_id = d.client_id
JOIN
  TRANS AS t
  ON d.account_id = t.account_id
WHERE
  SUBSTR(t.date, 1, 2) = '98' -- Filtra transacciones del año 1998 (asumiendo formato 'YYMMDD')
  AND c.client_id NOT IN (
    SELECT DISTINCT client_id
    FROM DISPOSITION
    WHERE account_id IN (SELECT DISTINCT account_id FROM LOAN)
  ) -- Excluye clientes que tienen un préstamo
GROUP BY
  c.client_id
HAVING
  COUNT(DISTINCT t.trans_id) > 10; -- Filtra clientes con más de 10 transacciones en 1998

 * sqlite:///data_berka.db
Done.


client_id,total_transactions_1998
1,62
4,92
5,92
6,69
7,66
8,61
9,70
10,85
11,85
12,56


2.7) Segmentar a los clientes en función al saldo de su cuenta (por ejemplo, bajo, medio, alto) y analizar la distribución de solicitudes de préstamos en estos segmentos.


In [30]:
%%sql

WITH AccountBalances AS (
  -- Calculate the current balance for each account
  SELECT
    t.account_id,
    COALESCE(SUM(CASE
                    WHEN t.type = 'PRIJEM' THEN t.amount
                    WHEN t.type = 'VYDAJ' THEN -t.amount
                    ELSE 0
                  END), 0) AS current_balance
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
),
RankedBalances AS (
  -- Rank accounts by balance to determine percentiles for segmentation
  SELECT
    account_id,
    current_balance,
    NTILE(3) OVER (ORDER BY current_balance) AS balance_segment_raw -- Divides into 3 roughly equal groups
  FROM
    AccountBalances
),
SegmentedAccounts AS (
  -- Assign meaningful names to the balance segments
  SELECT
    account_id,
    current_balance,
    CASE
      WHEN balance_segment_raw = 1 THEN 'Bajo'
      WHEN balance_segment_raw = 2 THEN 'Medio'
      WHEN balance_segment_raw = 3 THEN 'Alto'
      ELSE 'Desconocido'
    END AS balance_segment
  FROM
    RankedBalances
)
-- Main query to analyze loan distribution by balance segment
SELECT
  sa.balance_segment,
  COUNT(DISTINCT sa.account_id) AS total_accounts_in_segment,
  COUNT(L.loan_id) AS loan_applications_in_segment,
  ROUND(CAST(COUNT(L.loan_id) AS REAL) * 100 / COUNT(DISTINCT sa.account_id), 2) AS percentage_with_loan
FROM
  SegmentedAccounts AS sa
LEFT JOIN
  LOAN AS L
  ON sa.account_id = L.account_id
GROUP BY
  sa.balance_segment
ORDER BY
  CASE sa.balance_segment
    WHEN 'Bajo' THEN 1
    WHEN 'Medio' THEN 2
    WHEN 'Alto' THEN 3
    ELSE 4
  END;

 * sqlite:///data_berka.db
Done.


balance_segment,total_accounts_in_segment,loan_applications_in_segment,percentage_with_loan
Bajo,1500,76,5.07
Medio,1500,140,9.33
Alto,1500,466,31.07


2.8) Identifique a todos los clientes que incumplieron un préstamo.


In [31]:
%%sql
SELECT
  c.client_id,
  d.account_id,
  l.loan_id,
  l.date AS loan_start_date,
  l.amount AS loan_amount,
  l.status AS loan_status
FROM
  CLIENT AS c
JOIN
  DISPOSITION AS d
  ON c.client_id = d.client_id
JOIN
  LOAN AS l
  ON d.account_id = l.account_id
WHERE
  l.status IN ('B', 'D'); -- Filtra por estados de incumplimiento


 * sqlite:///data_berka.db
Done.


client_id,account_id,loan_id,loan_start_date,loan_amount,loan_status
25,19,4961,960429,30276,B
45,37,4967,981014,318480,D
124,103,4988,971206,265320,D
424,347,5045,950501,187224,B
512,426,5060,940719,252060,D
533,442,5063,970710,357840,D
568,472,5072,970428,196800,D
946,790,5126,940724,208128,B
968,808,5128,960411,215616,D
1327,1106,5176,941112,192552,B


2.9) Identifique cualquier característica común (como saldo en cuenta, volumen de transacciones, etc...) entre los clientes que incumplieron.

In [32]:
%%sql

WITH DefaultingClients AS (
  -- Identifica los client_id de aquellos que incumplieron un préstamo
  SELECT DISTINCT
    d.client_id
  FROM
    DISPOSITION AS d
  JOIN
    LOAN AS l
    ON d.account_id = l.account_id
  WHERE
    l.status IN ('B', 'D')
),
AccountBalances AS (
  -- Calcula el saldo actual de cada cuenta
  SELECT
    t.account_id,
    COALESCE(SUM(CASE
                    WHEN t.type = 'PRIJEM' THEN t.amount
                    WHEN t.type = 'VYDAJ' THEN -t.amount
                    ELSE 0
                  END), 0) AS current_balance
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
),
ClientTransactionCounts AS (
  -- Cuenta el número total de transacciones por cuenta
  SELECT
    t.account_id,
    COUNT(t.trans_id) AS total_transactions
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
)
SELECT
  -- Promedios de las características para los clientes que incumplieron
  AVG(AB.current_balance) AS average_balance_for_defaulters,
  AVG(CTC.total_transactions) AS average_transactions_for_defaulters
FROM
  DefaultingClients AS DC
JOIN
  DISPOSITION AS D
  ON DC.client_id = D.client_id
LEFT JOIN
  AccountBalances AS AB
  ON D.account_id = AB.account_id
LEFT JOIN
  ClientTransactionCounts AS CTC
  ON D.account_id = CTC.account_id;

 * sqlite:///data_berka.db
Done.


average_balance_for_defaulters,average_transactions_for_defaulters
193804.9026315789,299.4736842105263


2.10) Identifique la información que podría ayudar al banco a evaluar el riesgo de préstamos en futuras aplicaciones.

In [33]:
%%sql

WITH DefaultingClients AS (
  -- 1. Identifica los client_id de todos los clientes que incumplieron un préstamo
  -- Un préstamo se considera incumplido si su estado es 'B' (terminado con impago) o 'D' (en curso con impago).
  SELECT DISTINCT
    d.client_id
  FROM
    DISPOSITION AS d
  JOIN
    LOAN AS l
    ON d.account_id = l.account_id
  WHERE
    l.status IN ('B', 'D')
),
AccountBalances AS (
  -- 2. Calcula el saldo actual de cada cuenta
  -- Sumamos las transacciones: 'PRIJEM' (ingreso) suma, 'VYDAJ' (retiro) resta.
  SELECT
    t.account_id,
    COALESCE(SUM(CASE
                    WHEN t.type = 'PRIJEM' THEN t.amount
                    WHEN t.type = 'VYDAJ' THEN -t.amount
                    ELSE 0 -- En caso de otros tipos de transacciones no previstos
                  END), 0) AS current_balance
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
),
ClientTransactionCounts AS (
  -- 3. Cuenta el número total de transacciones por cuenta
  SELECT
    t.account_id,
    COUNT(t.trans_id) AS total_transactions
  FROM
    TRANS AS t
  GROUP BY
    t.account_id
)
SELECT
  -- 4. Calcula los promedios de saldo y transacciones para los clientes que incumplieron
  AVG(AB.current_balance) AS average_balance_for_defaulters,
  AVG(CTC.total_transactions) AS average_transactions_for_defaulters
FROM
  DefaultingClients AS DC
JOIN
  DISPOSITION AS D
  ON DC.client_id = D.client_id
LEFT JOIN
  AccountBalances AS AB
  ON D.account_id = AB.account_id
LEFT JOIN
  ClientTransactionCounts AS CTC
  ON D.account_id = CTC.account_id;

 * sqlite:///data_berka.db
Done.


average_balance_for_defaulters,average_transactions_for_defaulters
193804.9026315789,299.4736842105263
